### This notebook contains the routines for creating AMPTE_CHEM_nocal_pha.h5, AMPTE_CHEM_rates_dst.h5, AMPTE_CHEM_me.h5 and AMPTE_CHEM_hk_dst.h5 files for ampte_chem_webapp.  Created Dec 5, 2022

In [1]:
import datetime as dt
import multiprocessing as mp
import subprocess as sp
import time
from pathlib import Path

import numpy as np
import pandas as pd
from ampte_fits import read_fits_file_no_cal, read_fits_rates, \
    get_hk, add_kp_dst, calculate_range_counts, get_me, read_fits_file_only_cal

# fork is needed for macos
mp.set_start_method('fork')
_num_processes = 10

# set up paths for raw and processed data files
_apl_data_files = Path("./apl_data_files")
if not _apl_data_files.exists():
    _apl_data_files.mkdir()
_hdf_data_files = Path("./awa_data")
if not _hdf_data_files.exists():
    _hdf_data_files.mkdir()


### Create a hdf file containing PHA data with some housekeeping values in the index


In [ ]:
def create_ampte_chem_phas_nocal(files: list[Path]=None,
                                 h5file: Path= _hdf_data_files / 'AMPTE_CHEM_nocal_pha.h5'):
    """
    Create a blosc compressed hdf5 file containing only spins with cshk == 0,
    i.e., only non-calibration times, and without the time period
    1984:319:19:30 to 1984:325:00:18 with strange phas with a MultiIndex containing
    SpinStartTime, PAPS_lvl, DPU_Mode, TAC_Slope, PHA_Priority, L, MLTH, Voltage_Step,
    and Sector data
    """
    files = files or sorted(_apl_data_files.glob('198*gz'))
    try:
        # remove days 320 - 325 in 1984 with unusual PHAs
        for doy in range(320, 326):
            files.remove(_apl_data_files / f'data/1984_{doy}.fits.gz')
    except ValueError:
        pass
    h5file.unlink(missing_ok=True)
    pool_chunk_size = 40
    print(f'{pool_chunk_size} {dt.datetime.now():%F %T}')

    data_cols = ['SpinStartTime', 'PAPS_lvl', 'DPU_Mode', 'TAC_Slope', 'PHA_Priority',
                 'L', 'MLTH', 'Voltage_Step', 'Sector', 'SSDID', 'Energy']
    with pd.HDFStore(h5file, expectedrows=1_019_010_000, mode='w', complib='blosc:zstd') as store:
        pool = mp.Pool(processes=8)
        for i in range(0, len(files), pool_chunk_size):
            mapper = pool.map(read_fits_file_no_cal, files[i:min(len(files),i+pool_chunk_size)])
            print(f'{dt.datetime.now():%F %T}: appending data from files {i}-{i+pool_chunk_size} '
                  f'of {len(files)}/{pool_chunk_size}')
            store.append(key='ampte_pha',
                         value=pd.concat(mapper).sort_index(),
                         complib='blosc:zstd',
                         data_columns=data_cols,
                         index=False)
            print(f'{dt.datetime.now():%F %T} appending finished, '
                  f'{i} of {len(files)}/{pool_chunk_size}, {files[i].name}-{files[min(len(files),i+pool_chunk_size)-1].name}, '
                  f'{h5file.stat().st_size:_}')
        pool.close()
    time.sleep(5)

    print(f'{dt.datetime.now():%F %T}: creating table indexes')
    with pd.HDFStore(h5file, complib='blosc:zstd', expectedrows=1_019_010_000) as store:
            # index on SpinStartTime and Energy
            store.create_table_index(key='ampte_pha', optlevel=9, kind='full', columns=['SpinStartTime', 'Energy'])
    print(f'{dt.datetime.now():%F %T}: index created {h5file.stat().st_size:_}')


In [ ]:
def create_ampte_chem_cal_phas(h5file: Path=_hdf_data_files / 'AMPTE_CHEM_cal_pha.h5'):
    """
    Create a blosc compressed hdf5 file containing only spins with cshk > 0,
    i.e., only calibration times, and including the time period
    1984:319:19:30 to 1984:325:00:18 with unusual phas with a MultiIndex containing
    SpinStartTime, PAPS_lvl, DPU_Mode, TAC_Slope, PHA_Priority, L, MLTH, Voltage_Step,
    and Sector data
    """
    files = sorted(_apl_data_files.glob('198*fits.gz'))
    h5file.unlink(missing_ok=True)
    pool_chunk_size = 40
    print(f'{pool_chunk_size} {dt.datetime.now():%F %T}')

    data_cols = ['SpinStartTime', 'PAPS_lvl', 'DPU_Mode', 'TAC_Slope', 'PHA_Priority',
                 'L', 'MLTH', 'Voltage_Step', 'Sector', 'SSDID', 'Energy']
    with pd.HDFStore(h5file, expectedrows=1_019_010_000, mode='w', complib='blosc:zstd') as store:
        pool = mp.Pool(processes=_num_processes)
        for i in range(0, len(files), pool_chunk_size):
            mapper = pool.map(read_fits_file_only_cal, files[i:min(len(files),i+pool_chunk_size)])
            print(f'{dt.datetime.now():%F %T}: appending data from files {i}-{i+pool_chunk_size} of {len(files)}/{pool_chunk_size}')
            store.append(key='ampte_pha',
                         value=pd.concat(mapper).sort_index(),
                         complib='blosc:zstd',
                         data_columns=data_cols,
                         index=False)
            print(f'{dt.datetime.now():%F %T} appending finished, '
                  f'{files[i].name}-{files[min(len(files),i+pool_chunk_size)-1].name}, '
                  f'{h5file.stat().st_size:_}')
        pool.close()
    time.sleep(5)

    with pd.HDFStore(h5file, complib='blosc:zstd', expectedrows=1_019_010_000) as store:
        # index on SpinStartTime and Energy
        store.create_table_index(key='ampte_pha', optlevel=9, kind='full',
                                 columns=['SpinStartTime', 'Energy'])
    print(f'{dt.datetime.now():%F %T}: index created {h5file.stat().st_size:_}')


<hr>

### Create hdf file containing housekeeping, dst and kp data with every 32nd Spin Start Time values as the index

In [ ]:
def create_ampte_chem_hk_dst_file(files: list[Path] = None,
                                  h5file: Path = _hdf_data_files / 'AMPTE_CHEM_hk.h5'):
    """
    Create a blosc compressed hdf5 file containing SpinStartTime, housekeeping and space
    environment data with a RangeIndex
    """
    files = files or sorted(_apl_data_files.glob('198*fits.gz'))
    h5file.unlink(missing_ok=True)
    ndxcols = ['SpinStartTime', 'PAPS_lvl', 'DPU_Mode', 'TAC_Slope',
               'PHA_Priority', 'L', 'MLTH', 'MAGLAT', 'MAGLON']
    pool_chunk_size = 120
    key = 'ampte_hk'
    print(f'{pool_chunk_size} {dt.datetime.now():%F %T}')
    with pd.HDFStore(h5file, mode='w', complib='blosc:zstd') as store:
        pool = mp.Pool(processes=_num_processes)
        for i in range(0, len(files), pool_chunk_size):
            mapper = pool.map(get_hk, files[i:min(len(files), i + pool_chunk_size)])
            print(f'{dt.datetime.now():%F %T}: appending data from files '
                  f'{i}-{i+pool_chunk_size} of {len(files)}/{pool_chunk_size}')
            store.append(key=key,
                         value=pd.concat(mapper)
                                 .sort_values(by=['SpinStartTime', 'L'])
                                 .reset_index(drop=True),
                         data_columns=True,
                         complib='blosc:zstd',
                         index=False)
            print(f'{dt.datetime.now():%F %T} appending finished, '
                  f'{files[i].name}-{files[min(len(files),i+pool_chunk_size)-1].name}, '
                  f'{h5file.stat().st_size:_}')
        pool.close()
    time.sleep(3)
    print(f'{dt.datetime.now():%F %T}: {h5file.stat().st_size:_} creating table index')
    with pd.HDFStore(h5file, complib='blosc:zstd') as store:
        store.create_table_index(key=key, optlevel=9, columns=ndxcols, kind='full')
    print(f'{dt.datetime.now():%F %T}: index created, {h5file.stat().st_size:_}')
    add_kp_dst(h5file=h5file, key=key)
    # chunk size 60 1_585_322, 90 1_585_335, 30 1_589_752 | 120 20_612_973 int64, 120 20494236 int32


<hr>

### Create a hdf data file containing basic rate, br0-2 and R0-3 and housekeeping data with an rangeindex

In [2]:
def create_ampte_chem_rates_dst_file(files: list[Path]=None,
                                     h5file: Path=_hdf_data_files / 'AMPTE_CHEM_rates.h5'):
    """
    Create a hdf5 file containing SpinStartTime, some housekeeping and space environment
    data and rate data with a RangeIndex
    """
    def set_rate_col_types(_df: pd.DataFrame) -> pd.DataFrame:
        int8cols = ['DPU_Mode', 'PAPS_lvl', 'TAC_Slope', 'PHA_Priority', 'Period_Cntr', 'Voltage_Step']
        _df[int8cols] = _df[int8cols].astype('int8')
        int32cols = ['SSD', 'FSR', 'DCR', 'TCR', 'R0', 'R1', 'R2', 'BR0', 'BR1', 'BR2', ]
        _df[int32cols] = _df[int32cols].astype('int32')
        float32cols = ['L', 'MAGLAT', 'MAGLON']
        _df[float32cols] = _df[float32cols].astype('float32')
        return _df

    h5file.unlink(missing_ok=True)
    rates_dst_file = h5file.with_name(h5file.name.replace('rates', 'rates_dst'))
    rates_dst_file.unlink(missing_ok=True)
    repack_file = _hdf_data_files / 'AMPTE_CHEM_rates_dst_repack.h5'
    repack_file.unlink(missing_ok=True)
    files = files or sorted(_apl_data_files.glob('198*fits.gz'))
    pool_chunk_size = 40  # 120  can't use E>0 filter so 120 days gets too many PHAs
    print(f'{pool_chunk_size} {dt.datetime.now():%F %T}')
    # with pd.HDFStore(h5file, complib='blosc:zstd', expectedrows=20_255_351) as store:
    with pd.HDFStore(h5file, mode='w', expectedrows=20_600_000) as store:
        pool = mp.Pool(processes=_num_processes)
        for i in range(0, len(files), pool_chunk_size):
            mapper = pool.map(read_fits_rates, files[i:min(len(files), i+pool_chunk_size)])
            print(f'{dt.datetime.now():%F %T}: appending data from files {i}-{i+pool_chunk_size} of {len(files)}/{pool_chunk_size}')
            # ratesdf = pd.concat(mapper)
            store.append(key='ampte_rates',
                         value=set_rate_col_types(pd.concat(mapper)),
                         # value=set_rate_col_types(ratesdf),
                         # value=(ratesdf
                         #          .merge(calculate_range_counts(_df=ratesdf),
                         #                 on='SpinStartTime', how='outer')
                         #          .fillna(0)
                         #          .pipe(set_rate_col_types)
                         #        ),
                         # complib='blosc:zstd',
                         data_columns=True,
                         index=False)
            print(f'{dt.datetime.now():%F %T} appending finished, '
                  f'{i} of {len(files)}/{pool_chunk_size}, '
                  f'{files[i].name}-{files[min(len(files), i + pool_chunk_size) - 1].name}, '
                  f'{h5file.stat().st_size:_}')
        pool.close()
    time.sleep(5)
    add_kp_dst(h5file=h5file, key='ampte_rates')
    print(f'{dt.datetime.now():%F %T}: kp and dst added')
    # use ptrepack to save a copy of the h5 file sorted by SpinStartTime
    cmd = ('ptrepack --sortby SpinStartTime  --checkCSI --chunkshape auto '
           f'{rates_dst_file} {repack_file}')
    sp.call(cmd.split())

    # ndxcols = ['SpinStartTime', 'PAPS_lvl', 'DPU_Mode', 'TAC_Slope', 'FSR',
    #            'PHA_Priority', 'L', 'MLTH', 'MAGLAT', 'MAGLON', 'Period_Cntr',
    #            'Voltage_Step', 'FSR']

    # Testing shows that indexing several columns slows down reads with a where clause
    ndxcols = ['SpinStartTime']
    print(f'{dt.datetime.now():%F %T}: creating indexes')
    with pd.HDFStore(repack_file, expectedrows=20_600_000, ) as store:
        store.create_table_index(key='ampte_rates', optlevel=9, kind='full', columns=ndxcols)
    print(f'{dt.datetime.now():%F %T}: index created {h5file.stat().st_size:_}')
    repack_file.unlink(missing_ok=True)


create_ampte_chem_rates_dst_file()

<hr>

### Create a hdf data file containing matrix element data

In [ ]:
def create_ampte_chem_me_file(files: list[Path] = None,
                              h5file: Path = _hdf_data_files / 'AMPTE_CHEM_me.h5'):
    """
    Create a blosc compressed hdf5 file with a DateTimeIndex named SpinStartTime
    containing L values and matrix elements
    """
    files = files or sorted(_apl_data_files.glob('198*fits.gz'))
    h5file.unlink(missing_ok=True)
    pool_chunk_size = 120
    print(f'{pool_chunk_size} {dt.datetime.now():%F %T}')
    with pd.HDFStore(h5file, complib='blosc:zstd') as store:
        pool = mp.Pool(processes=6)
        for i in range(0, len(files), pool_chunk_size):
            mapper = pool.map(get_me, files[i:min(len(files),i+pool_chunk_size)])
            print(f'{dt.datetime.now():%F %T}: appending data from files '
                  f'{i}-{i+pool_chunk_size} of {len(files)}/{pool_chunk_size}')
            store.append(key='ampte_me',
                         value=pd.concat(mapper)
                                 .sort_values(by=['SpinStartTime', 'L']),
                         data_columns=['L'],
                         complib='blosc:zstd',
                         index=False,
                         chunksize=512,
                        )
            print(f'{dt.datetime.now():%F %T} appending finished, '
                  f'{files[i].name}-{files[min(len(files),i+pool_chunk_size)-1].name}, '
                  f'{h5file.stat().st_size:_}')
        pool.close()
    time.sleep(3)
    print(f'{dt.datetime.now():%F %T}: {h5file.stat().st_size:_} creating table index')
    with pd.HDFStore(h5file, complib='blosc:zstd') as store:
        store.create_table_index(key='ampte_me', optlevel=9, columns=['L'], kind='full')
    print(f'{dt.datetime.now():%F %T}: index created, {h5file.stat().st_size:_}')


create_ampte_chem_me_file()

## Information and the first few rows of the hdf files created above

In [7]:
pd.read_hdf(_hdf_data_files / 'AMPTE_CHEM_me.h5', 'ampte_me', start=0, stop=15).info()
pd.read_hdf(_hdf_data_files / 'AMPTE_CHEM_me.h5', 'ampte_me', start=0, stop=15).iloc[:, [0,1,2,3,4,5,510,511, 512]]


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 15 entries, 1966-01-01 00:00:05 to 1984-08-21 17:58:06
Columns: 513 entries, L to 511
dtypes: float32(1), int32(512)
memory usage: 30.2 KB


,L,0,1,2,3,4,509,510,511
SpinStartTime,,,,,,,,,
1966-01-01 00:00:05,1.469368e-39,2048,1344,1344,2560,1664,20480,22528,248
1984-08-21 17:20:34,1.469368e-39,0,0,0,0,0,92,23552,320
1984-08-21 17:22:15,7.266188e+00,0,0,0,0,0,96,23552,320
1984-08-21 17:25:14,7.221412e+00,0,0,0,0,0,96,23552,320
1984-08-21 17:28:13,7.175906e+00,0,0,0,0,0,88,23552,336
1984-08-21 17:31:13,7.129500e+00,0,0,0,0,0,92,23552,336
1984-08-21 17:34:12,7.082700e+00,0,0,0,0,0,96,23552,336
1984-08-21 17:37:11,7.034521e+00,0,0,0,0,0,96,23552,352
1984-08-21 17:40:10,6.986234e+00,0,0,0,0,0,96,23552,352


In [9]:
pd.read_hdf(_hdf_data_files / 'AMPTE_CHEM_hk_dst.h5', key='ampte_hk', start=0, stop=15).info()
pd.read_hdf(_hdf_data_files / 'AMPTE_CHEM_hk_dst.h5', key='ampte_hk', start=0, stop=15)


<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 0 to 14
Data columns (total 10 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SpinStartTime  15 non-null     datetime64[ns]
 1   PAPS_lvl       15 non-null     int32         
 2   DPU_Mode       15 non-null     int8          
 3   TAC_Slope      15 non-null     int8          
 4   PHA_Priority   15 non-null     int8          
 5   L              15 non-null     float32       
 6   MAGLAT         15 non-null     float32       
 7   MAGLON         15 non-null     float32       
 8   Dst            14 non-null     float64       
 9   Kp             14 non-null     float64       
dtypes: datetime64[ns](1), float32(3), float64(2), int32(1), int8(3)
memory usage: 765.0 bytes


,SpinStartTime,PAPS_lvl,DPU_Mode,TAC_Slope,PHA_Priority,L,MAGLAT,MAGLON,Dst,Kp
0,1966-01-01 00:00:05,5,2,0,78,1.469368e-39,0.103796,2.581452,NaN,NaN
1,1984-08-21 17:20:34,-1,0,0,0,1.469368e-39,0.090201,4.181586,2.0,0.333
2,1984-08-21 17:22:15,-1,0,0,78,7.266188e+00,0.090361,4.183877,2.0,0.333
3,1984-08-21 17:25:14,-1,0,0,78,7.221412e+00,0.090761,4.189670,2.0,0.333
4,1984-08-21 17:28:13,-1,0,0,78,7.175906e+00,0.091303,4.197631,2.0,0.333
5,1984-08-21 17:31:13,-1,0,0,78,7.129500e+00,0.091826,4.206097,2.0,0.333
6,1984-08-21 17:34:12,-1,0,0,78,7.082700e+00,0.092166,4.211849,2.0,0.333
7,1984-08-21 17:37:11,-1,0,0,78,7.034521e+00,0.092617,4.220331,2.0,0.333
8,1984-08-21 17:40:10,-1,0,0,78,6.986234e+00,0.092908,4.225976,2.0,0.333
9,1984-08-21 17:43:10,-1,0,0,78,6.936394e+00,0.093319,4.234886,2.0,0.333


In [10]:
pd.read_hdf(_hdf_data_files / 'AMPTE_CHEM_rates_dst.h5', key='ampte_rates', start=0, stop=15).info()
pd.read_hdf(_hdf_data_files / 'AMPTE_CHEM_rates_dst.h5', key='ampte_rates', start=0, stop=15)


<class 'pandas.core.frame.DataFrame'>
Index: 15 entries, 0 to 14
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   SpinStartTime  15 non-null     datetime64[ns]
 1   PAPS_lvl       15 non-null     int8          
 2   DPU_Mode       15 non-null     int8          
 3   TAC_Slope      15 non-null     int8          
 4   PHA_Priority   15 non-null     int8          
 5   L              15 non-null     float32       
 6   MAGLAT         15 non-null     float32       
 7   MAGLON         15 non-null     float32       
 8   Period_Cntr    15 non-null     int8          
 9   Voltage_Step   15 non-null     int8          
 10  FSR            15 non-null     int32         
 11  DCR            15 non-null     int32         
 12  TCR            15 non-null     int32         
 13  SSD            15 non-null     int32         
 14  BR0            15 non-null     int32         
 15  BR1            15 non-null    

,SpinStartTime,PAPS_lvl,DPU_Mode,TAC_Slope,PHA_Priority,L,MAGLAT,MAGLON,Period_Cntr,Voltage_Step,...,TCR,SSD,BR0,BR1,BR2,R0,R1,R2,Dst,Kp
0,1984-08-21 17:20:34,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333
1,1984-08-21 17:20:40,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333
2,1984-08-21 17:20:45,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333
3,1984-08-21 17:20:51,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333
4,1984-08-21 17:20:57,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333
5,1984-08-21 17:21:02,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333
6,1984-08-21 17:21:08,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333
7,1984-08-21 17:21:13,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333
8,1984-08-21 17:21:19,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333
9,1984-08-21 17:21:25,-1,0,0,0,1.469368e-39,0.090201,4.181586,0,0,...,0,0,0,0,0,0,0,0,2.0,0.333


In [12]:
pd.read_hdf(_hdf_data_files / 'AMPTE_CHEM_nocal_pha.h5', key='ampte_pha', start=0, stop=15).info()
pd.read_hdf(_hdf_data_files / 'AMPTE_CHEM_nocal_pha.h5', key='ampte_pha', start=0, stop=15)


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 15 entries, (Timestamp('1966-01-01 00:00:11'), np.int8(5), np.int8(2), np.int8(0), np.int8(78), np.float64(4.809903621673584), np.float32(9.860421), np.int8(0), np.int8(0), np.int8(2)) to (Timestamp('1966-01-01 00:00:11'), np.int8(5), np.int8(2), np.int8(0), np.int8(78), np.float64(4.809903621673584), np.float32(9.860421), np.int8(0), np.int8(4), np.int8(3))
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   Energy  15 non-null     int16
 1   TOF     15 non-null     int16
 2   Range   15 non-null     int8 
dtypes: int16(2), int8(1)
memory usage: 1.6 KB


Energy  \
SpinStartTime       PAPS_lvl DPU_Mode TAC_Slope PHA_Priority L        MLTH     Voltage_Step Sector SSDID           
1966-01-01 00:00:11 5        2        0         78           4.809904 9.860421 0            0      2           0   
                                                                                                   2           0   
                                                                                                   2           0   
                                                                                                   3           0   
                                                                                                   3           0   
                                                                                            1      2           0   
                                                                                                   2           0   
                                                                                            4      2           0   
                                                                                                   2         119   
                                                                                                   2           0   
                                                                                                   2           0   
                                                                                                   2           0   
                                                                                                   3           0   
                                                                                                   3           0   
                                                                                                   3           0   

                                                                                                           TOF  \
SpinStartTime       PAPS_lvl DPU_Mode TAC_Slope PHA_Priority L        MLTH     Voltage_Step Sector SSDID         
1966-01-01 00:00:11 5        2        0         78           4.809904 9.860421 0            0      2      1019   
                                                                                                   2        59   
                                                                                                   2        22   
                                                                                                   3       678   
                                                                                                   3       934   
                                                                                            1      2       139   
                                                                                                   2      1019   
                                                                                            4      2       211   
                                                                                                   2        54   
                                                                                                   2        22   
                                                                                                   2       112   
                                                                                                   2      1019   
                                                                                                   3        33   
                                                                                                   3       995   
                                                                                                   3      1019   

                                                                                                          Range  
SpinStartTime       PAPS_lvl DPU_Mode TAC_Slope PHA_Priority L        MLTH     V